In [173]:
import numpy as np
from scipy.interpolate import BarycentricInterpolator
from scipy.integrate import quad
from chebyshev_interpolator import ChebyshevInterpolator
from Option import OptionType
from quadrature_nodes import QuadratureNodes
from QDPlus import QDplus

In [174]:
## Define Variables to being with
l = 5
n =5
max_tau =1 
strike = 100
riskfree = 0.025
dividend = 0.025
volatility = 0.25
option_type = OptionType.Put
X = strike*min(1,riskfree/dividend)


In [ ]:
## 1. Obtain Chebyshev nodes to begin
cheby = ChebyshevInterpolator(n,1)
cheby.compute_nodes()
tau_nodes = cheby.get_nodes()[1]

## Fill up rest of the variables
initial_boundary = np.zeros(n+1)

## print tau_nodes
tau_nodes

array([0.        , 0.00911863, 0.11936438, 0.42838137, 0.81813562,
       1.        ])

In [176]:
## 2. compute the Quadrature nodes and weights
quadrature = QuadratureNodes(l)
quadrature.compute_legendre_nodes()
y_nodes, w_weights = quadrature.get_nodes_and_weights()

In [296]:
## 3. compute initial boundary using QD Plus
a=QDplus(riskfree, dividend, volatility, strike, option_type)
for i in range(len(tau_nodes)):
    initial_boundary[i] = a.compute_exercise_boundary(tau_nodes[i])

## print intial_boundary
print(cheby.z_nodes)
print(tau_nodes)
print(initial_boundary)

[-1.         -0.80901699 -0.30901699  0.30901699  0.80901699  1.        ]
[0.         0.00911863 0.11936438 0.42838137 0.81813562 1.        ]
[100.          92.55238125  80.23115496  70.10034581  64.05178323
  62.08739664]


In [ ]:
## compute q(_initial_boundary), since H_x = q_z 
def B_to_q(_initial_boundary):
    """
    Compute H(sqrt(tau)) for each collocation point based on the previous boundary values.

    Returns:
    - H_values (numpy array): The computed H(sqrt(tau)) values.
    """
    q_values = (np.log(_initial_boundary / X)) ** 2 

    return q_values

## compute b(_q_values) to obtain the reverse 
def q_to_B(_q_values):
    """
    Given that H_values = q
    From q/H_values calculate B

    Returns:
    - B_values (numpy array): The computed B(tau) values.
    """
    B_values = np.zeros(len(_q_values))
    for i in range(len(_q_values)):
        if _q_values[i] < 1: ### When q_value is less than 1 it means that log(B(tau)/X) was negative value
            B_values[i] = X * np.exp(-np.sqrt(_q_values[i]))
        else:
            B_values[i] = X * np.exp(np.sqrt(_q_values[i]))

    return B_values



In [294]:
## Testing for B to q and then from q to B
print(q_to_B(B_to_q(initial_boundary)))
print(initial_boundary)

[100.          92.55238125  80.23115496  70.10034581  64.05178323
  62.08739664]
[100.          92.55238125  80.23115496  70.10034581  64.05178323
  62.08739664]


In [287]:
## 4.1 initialize_chebyshev_interpolation to compute coefficients a_k
"""
Initialize the Chebyshev interpolation by computing the coefficients a_k.
"""
def Cheby_interpolant(_q):
    """
    Initialize the Chebyshev interpolation by computing the coefficients a_k.
    
    Parameters:
    - _q (numpy array): The function values at Chebyshev nodes.
    
    Returns:
    - a_coefficients (numpy array): The computed Chebyshev coefficients a_k.
    """
    n = len(_q) - 1  # Assuming _q has n+1 elements
    a_coefficients = np.zeros(n + 1)
    
    term1 = (1/(2*n))*(_q[0]+((-1)**n)*_q[n])

    for k in range(n + 1):
        sum_value = 0.0
        for i in range(1, n):  # This ranges from 1 to n-1
            cos_term = np.cos(i * k * np.pi / n)
            sum_value += _q[i] * cos_term
            
        # Correct the formula to include q[n] in the a_k calculation
        a_coefficients[k] = term1 + (2 / n) * sum_value

    return a_coefficients

In [291]:
def clenshaw_algorithm(z, a_coefficients):
    """
    Evaluate the Chebyshev interpolant using the Clenshaw algorithm.

    Parameters:
    - z (float): The point at which to evaluate the interpolant.
    - a_coefficients (numpy array): The Chebyshev coefficients a_k.

    Returns:
    - qC (float): The evaluated Chebyshev interpolant at z.
    """
    n = len(a_coefficients) - 1  # n is the degree of the polynomial
    b_next = 0.0
    b_curr = a_coefficients[n]  # Initialize b_n(z)

    # Iterate from n-1 down to 1
    for k in range(n - 1, 0, -1):
        b_temp = b_curr
        b_curr = a_coefficients[k] + 2 * z * b_curr - b_next
        b_next = b_temp

    # Final computation of qC(z)
    return a_coefficients[0] + z * b_curr - b_next

In [293]:
### Test for Clenshaw for inpterolation by ensuring that I can obtain q(zi) for the given zi of the intial boundary 

q = B_to_q(initial_boundary) # obtain the q from intial boundary
z_i = cheby.z_nodes # obtain the z_i that we will try to interpolate

test_a_coefficients = Cheby_interpolant(B_to_q(initial_boundary)) # obtain a_coefficients that will be used for interpolation

for i in range(len(z_i)):
    print(q[i], clenshaw_algorithm(z_i[i], test_a_coefficients))

0.0 0.052874902113678235
0.0059900507192958115 0.27451944125613753
0.04851371024554126 0.1790721066785782
0.12619720456489988 0.12458222071011807
0.19845093079156076 0.05886495283297416
0.2271734595026403 -0.03751821928674336


In [76]:
## 4.3 Def Interpolation scheme to be used for quadrature later
# This means that we would be obtaining for tau - tau(1+y)**2/4

def evaluate_boundary(tau_nodes, y_nodes, a_coefficients):
    """
    For each tau_i, evaluate B(tau + y) using Clenshaw algorithm at the adjusted points.

    Parameters:
    - tau_nodes (numpy array): Array of tau values.
    - y_nodes (numpy array): Array of y values.
    - a_coefficients (list): Coefficients for the Clenshaw algorithm.

    Returns:
    - B_values_list (list of numpy arrays): Evaluated boundary values for each tau.
    """
    B_values_list = []

    ## For each tau we hope to obtain a series of nodes
    for tau in tau_nodes:
        qz_interpolated = np.zeros(len(y_nodes))
        
        # Use the quadrature nodes obtained in the quadacture earlier
        for k, y_k in enumerate(y_nodes):
            adjusted_tau = max(tau - tau * (1 + y_k)**2 / 4, 0)               # Obtain the adjusted tau for each y_k, but the adjusted tau cannot be negative
            z = 2 * np.sqrt(adjusted_tau / np.max(tau_nodes)) - 1             # From the tau obtain the z value
            qz_interpolated[k] = max(clenshaw_algorithm(z, a_coefficients),0) # Interpolate qz using the clenshaw algorithm

        B_values = np.exp(qz_interpolated**(0.5))*strike                      # q(z) = H(xi) to back out B_value for the z, here just used Strike for X as we assume r=q

        B_values_list.append(B_values)

    return B_values_list

In [75]:
integral_B_Values = evaluate_boundary(tau_nodes, y_nodes, a_coefficients)
integral_B_Values

0.0
0.0
0.0
0.0
0.0
0.009098561065933996
0.008633036102261297
0.0068389703320460345
0.0037229352535545473
0.0008354449567705719
0.11910171059665842
0.11300790970912497
0.0895232838867434
0.04873385515860508
0.010936116462775217
0.4274386959784934
0.4055689319602345
0.32128602966795394
0.17489870963770493
0.0392481294894913
0.8163352687842573
0.7745677361390804
0.6136017161132566
0.3340268124655224
0.07495725735064318
0.9977994446729768
0.946747355571419
0.75
0.408278045465736
0.09161959873431291


[array([123.35676519, 123.35676519, 123.35676519, 123.35676519,
        123.35676519]),
 array([142.89933987, 142.44275736, 140.52175518, 136.27342141,
        129.61591732]),
 array([167.68494871, 167.59122094, 166.57923975, 161.01105344,
        144.56711327]),
 array([144.96532164, 146.87192371, 154.66215102, 166.37542234,
        158.54146404]),
 array([135.681186  , 133.93371883, 133.50410771, 153.46580998,
        165.28717512]),
 array([148.5499542 , 144.26474693, 133.19335292, 146.63188336,
        166.71813952])]

In [70]:
y_nodes

array([-0.90617985, -0.53846931,  0.        ,  0.53846931,  0.90617985])

In [ ]:
# calculate K1 integrad for each tau[i] to be used for quadrature np.sum(weights*integrads) 
# Need to define d_plus

def K1_integrad(tau, B_tau, y, B_y):
    term1 = np.exp((-dividend/4)*tau[i]*(1+y[i])**2)*(y[i]+1)
    term2 = scipy.stats.norm(d_plus(tau[i]*(1+y[i])**2/4, B_tau[i]/B_y[i]))
    return term1*term2

In [ ]:
# calculate K2 integrad for each tau[i] to be used for quadrature np.sum(weights*integrads) 

def K2_integrad(tau, B_tau, y, B_y):
    term1 = np.exp((-dividend/4)*tau[i]*(1+y[i])**2)*(y[i]+1)/sigma
    term2 = scipy.stats.norm.cdf(d_plus(tau[i]*(1+y[i])**2/4, B_tau[i]/B_y[i]))
    return term1*term2

In [ ]:
# Need to define d_minus

def K3_integrad(tau, B_tau, y, B_y):
    term1 = np.exp((-dividend/4)*tau[i]*(1+y[i])**2)*(y[i]+1)/sigma
    term2 = scipy.stats.norm.cdf(d_plus(tau[i]*(1+y[i])**2/4, B_tau[i]/B_y[i]))
    return term1*term2